In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'TRUESessionwiseGRU'
device = 'cuda:2'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SessionwiseGRU(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        slate_size = shp[-1]
        item_embs, user_embs = self.embedding(batch)
        
        # item_embs: batch, session, slate, embedding
        item_embs = item_embs.flatten(-3, -2)
        hidden =  user_embs[None, ..., 0, :].contiguous()

        rnn_out, _ = self.rnn_layer(
            item_embs,
            hidden,
        )
        return self.out_layer(rnn_out).reshape(shp)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SessionwiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 2841.67it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.3333333134651184, 'accuracy': 0.25}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | treshold: 0.51
Test: accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | 
Val update: epoch: 2 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | treshold: 0.51
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | 
Val update: epoch: 3 |accuracy: 0.5 | f1: 0.5 | auc: 1.0 | treshold: 0.52
Test: accuracy: 0.5 | f1: 0.5 | auc: 1.0 | 
Val update: epoch: 4 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.53
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SessionwiseGRU(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (rnn_layer): GRU(32, 32, batch_first=True, dropout=0.1)
   (out_layer): Linear(in_features=32, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings
        ),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating TRUESessionwiseGRU with svd embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.1636270135641098, 'roc-auc': 0.4824480414390564, 'accuracy': 0.31032153964042664}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.0963149219751358 | f1: 0.1757066696882248 | auc: 0.5165818333625793 | treshold: 0.01
Test: accuracy: 0.10087374597787857 | f1: 0.18326124548912048 | auc: 0.5269505977630615 | 
Val update: epoch: 1 |accuracy: 0.0963149219751358 | f1: 0.1757066696882248 | auc: 0.5360655188560486 | treshold: 0.01
Test: accuracy: 0.10087374597787857 | f1: 0.18326124548912048 | auc: 0.5486103296279907 | 
Val update: epoch: 2 |accuracy: 0.8627267479896545 | f1: 0.19119800627231598 | auc: 0.5783790349960327 | treshold: 0.06999999999999999
Test: accuracy: 0.8606562614440918 | f1: 0.2106293886899948 | auc: 0.5850542783737183 | 
Val update: epoch: 3 |accuracy: 0.836736798286438 | f1: 0.2283937782049179 | auc: 0.6159082651138306 | treshold: 0.06999999999999999
Test: accuracy: 0.8302794098854065 | f1: 0.24316748976707458 | auc: 0.6196414232254028 | 
Val update: epoch: 4 |accuracy: 0.8549063801765442 | f1: 0.22610323131084442 | auc: 0.6307203769683838 | treshold: 0.12
Test: accurac

/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.18279023468494415, 'roc-auc': 0.5266836881637573, 'accuracy': 0.36824652552604675}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.0963149219751358 | f1: 0.1757066696882248 | auc: 0.6206378936767578 | treshold: 0.05
Test: accuracy: 0.10087374597787857 | f1: 0.18326124548912048 | auc: 0.6214832067489624 | 
Val update: epoch: 1 |accuracy: 0.8456096053123474 | f1: 0.21670126914978027 | auc: 0.648362398147583 | treshold: 0.09999999999999999
Test: accuracy: 0.8401326537132263 | f1: 0.24121727049350739 | auc: 0.653434157371521 | 
Val update: epoch: 2 |accuracy: 0.628363847732544 | f1: 0.24571293592453003 | auc: 0.6786197423934937 | treshold: 0.13
Test: accuracy: 0.6129741668701172 | f1: 0.25238296389579773 | auc: 0.6828413009643555 | 
Val update: epoch: 3 |accuracy: 0.8375407457351685 | f1: 0.26736980676651 | auc: 0.696010947227478 | treshold: 0.15000000000000002
Test: accuracy: 0.8315208554267883 | f1: 0.28109702467918396 | auc: 0.698336124420166 | 
Val update: epoch: 4 |accuracy: 0.8666588664054871 | f1: 0.2532743811607361 | auc: 0.7062798738479614 | treshold: 0.12
Test: accuracy: 0.8

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating TRUESessionwiseGRU with explicit embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.5870500206947327, 'roc-auc': 0.41889268159866333, 'accuracy': 0.48460888862609863}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7494558095932007 | f1: 0.7997602820396423 | auc: 0.8119739294052124 | treshold: 0.44
Test: accuracy: 0.7502115964889526 | f1: 0.801063060760498 | auc: 0.8148013353347778 | 
Val update: epoch: 1 |accuracy: 0.7542446851730347 | f1: 0.8345625996589661 | auc: 0.8271102905273438 | treshold: 0.39
Test: accuracy: 0.7582879662513733 | f1: 0.8378325700759888 | auc: 0.8280985355377197 | 
Val update: epoch: 2 |accuracy: 0.7855415344238281 | f1: 0.8444413542747498 | auc: 0.8417257070541382 | treshold: 0.37
Test: accuracy: 0.7864828705787659 | f1: 0.8457642197608948 | auc: 0.8438141345977783 | 
Val update: epoch: 3 |accuracy: 0.7637739777565002 | f1: 0.8402074575424194 | auc: 0.8429273962974548 | treshold: 0.33
Test: accuracy: 0.7672349214553833 | f1: 0.8432860374450684 | auc: 0.8412854671478271 | 
Val update: epoch: 4 |accuracy: 0.7784550189971924 | f1: 0.8454425930976868 | auc: 0.8506739139556885 | treshold: 0.38
Test: accuracy: 0.7811872959136963 | f1: 0.8479083

...

Val update: epoch: 0 |accuracy: 0.7263338565826416 | f1: 0.8199023008346558 | auc: 0.7916935086250305 | treshold: 0.39
Test: accuracy: 0.7295852899551392 | f1: 0.8228323459625244 | auc: 0.7917734384536743 | 
Val update: epoch: 1 |accuracy: 0.7810912728309631 | f1: 0.8387637138366699 | auc: 0.8325304388999939 | treshold: 0.38
Test: accuracy: 0.7780196070671082 | f1: 0.8369970321655273 | auc: 0.8320802450180054 | 
Val update: epoch: 2 |accuracy: 0.7672809958457947 | f1: 0.8413781523704529 | auc: 0.8373182415962219 | treshold: 0.37
Test: accuracy: 0.7701124548912048 | f1: 0.8438531756401062 | auc: 0.8365684747695923 | 
Val update: epoch: 3 |accuracy: 0.7800754904747009 | f1: 0.8455069065093994 | auc: 0.8429388999938965 | treshold: 0.38
Test: accuracy: 0.7807762026786804 | f1: 0.846474289894104 | auc: 0.8409845232963562 | 
Val update: epoch: 4 |accuracy: 0.7873312830924988 | f1: 0.8471394181251526 | auc: 0.8525184392929077 | treshold: 0.35000000000000003
Test: accuracy: 0.7866279482841492 

/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028



Evaluating TRUESessionwiseGRU with svd embeddings
Test before learning: {'f1': 0.0, 'roc-auc': 0.5316640138626099, 'accuracy': 0.35241204500198364}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.6406907439231873 | f1: 0.7810012102127075 | auc: 0.6423238515853882 | treshold: 0.01
Test: accuracy: 0.6475879549980164 | f1: 0.7861042618751526 | auc: 0.6422170400619507 | 
Val update: epoch: 1 |accuracy: 0.6406907439231873 | f1: 0.7810012102127075 | auc: 0.6632378697395325 | treshold: 0.01
Test: accuracy: 0.6475879549980164 | f1: 0.7861042618751526 | auc: 0.6666902303695679 | 
Val update: epoch: 2 |accuracy: 0.6406907439231873 | f1: 0.7810012102127075 | auc: 0.761383593082428 | treshold: 0.26
Test: accuracy: 0.6475879549980164 | f1: 0.7861042618751526 | auc: 0.7632458209991455 | 
Val update: epoch: 3 |accuracy: 0.6727615594863892 | f1: 0.7924146056175232 | auc: 0.7762966156005859 | treshold: 0.38
Test: accuracy: 0.679385781288147 | f1: 0.7974890470504761 | auc: 0.7775858044624329 | 
Val update: epoch: 4 |accuracy: 0.7309050559997559 | f1: 0.8097859621047974 | auc: 0.7963799238204956 | treshold: 0.41000000000000003
Test: accuracy: 0.7335993051528931 |

In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items